In [ ]:
import tensorflow as tf
from tensorflow import keras



In [ ]:
# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()



In [ ]:
# Preprocess the data
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

In [ ]:
# Convert the labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)



In [ ]:
# Define the Residual Block
def residual_block(inputs, filters):
    x = keras.layers.Conv2D(filters, 3, padding='same', activation='relu')(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(filters, 3, padding='same', activation=None)(x)
    x = keras.layers.BatchNormalization()(x)

    # Skip connection
    shortcut = keras.layers.Conv2D(filters, 1, padding='same')(inputs)
    x = keras.layers.add([x, shortcut])
    x = keras.layers.Activation('relu')(x)
    return x



In [ ]:
# Build the ResNet model
def create_resnet_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape)

    # Initial convolution layer
    x = keras.layers.Conv2D(32, 3, padding='same', activation='relu')(inputs)

    # Residual blocks
    x = residual_block(x, 32)
    x = residual_block(x, 32)

    x = keras.layers.MaxPooling2D(pool_size=2)(x)

    x = residual_block(x, 64)
    x = residual_block(x, 64)

    x = keras.layers.GlobalAveragePooling2D()(x)

    # Output layer
    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

    model = keras.models.Model(inputs=inputs, outputs=outputs)
    return model



In [ ]:
# Create the ResNet model
input_shape = (28, 28, 1)
num_classes = 10
model = create_resnet_model(input_shape, num_classes)



In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [ ]:
# Print model summary
model.summary()




In [ ]:
# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.1)



In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy:", test_acc)